1. Load the keras model 'mnist_model.h5', which was trained in mnist_keras code. <br />
2. convert it to tensorflow saved_model <br />
    Note that the 'image' in predict_signature_def will be used later. 
3. save the saved_model in mnist_model folder

In [ ]:
from __future__ import print_function

import keras
from keras import backend as K    
import tensorflow as tf 
from keras.models import load_model

In [13]:
model = load_model('mnist_model.h5')

In [14]:
previous_model = model

K.set_learning_phase(0)  # all new operations will be in test mode from now on

# serialize the model and get its weights, for quick re-building
config = previous_model.get_config()
weights = previous_model.get_weights()

# re-build a model where the learning phase is now hard-coded to 0
from keras.models import model_from_config
from keras.models import Sequential


new_model = Sequential.from_config(config)
#new_model = model_from_config(config)
new_model.set_weights(weights)

model = new_model

In [16]:
signature = tf.saved_model.signature_def_utils.predict_signature_def(                                                                        
    inputs={'image': model.input}, outputs={'scores': model.output})                                                                         
                                                                                                                                             
builder = tf.saved_model.builder.SavedModelBuilder('mnist_model')                                                                    
builder.add_meta_graph_and_variables(                                                                                                        
    sess=K.get_session(),                                                                                                                    
    tags=[tf.saved_model.tag_constants.SERVING],                                                                                             
    signature_def_map={                                                                                                                      
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:                                                                
            signature                                                                                                                        
    })                                                                                                                                       
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: mnist_model/saved_model.pb


'mnist_model/saved_model.pb'